In [1]:
from pymongo import MongoClient
from tqdm import tqdm
import numpy as np
import os
import pandas as pd
import pymongo
import xarray as xr

In [2]:
client = MongoClient()
db = client.eddies
collection = db.cchdo

In [3]:
def format(LON):
    if LON < 0:
        LON = LON + 360
    else:
        LON = LON
    return LON

In [4]:
rootdir = '/home/cz2397/data/cchdo-ctd/'

In [5]:
station = 0
overlap = 0
result = []

In [6]:
db.cchdo.create_index([('dat', pymongo.ASCENDING), ('loc', pymongo.ASCENDING)])

u'dat_1_loc_1'

In [7]:
for subdir, dirs, files in os.walk(rootdir):
    for file in tqdm(files):
        if file.endswith('ctd.nc'):
            url = os.path.join(subdir, file)
            DS = xr.open_dataset(url)
            DAT = DS.time.values[0]
            LAT = DS.latitude.values[0]
            LON = format(DS.longitude.values[0])
            dat_var = np.timedelta64(3, 'D')
            dat_min = pd.to_datetime(DAT - dat_var)
            dat_max = pd.to_datetime(DAT + dat_var)
            lat_var = 0.2
            lat_min = LAT - lat_var
            lat_max = LAT + lat_var
            lon_var = 0.4
            lon_min = LON - lon_var
            lon_max = LON + lon_var
            ft = {
                'dat': {'$gt': dat_min, '$lt': dat_max},
                'loc': {'$within': {'$box': [[lon_min, lat_min], [lon_max, lat_max]]}}
            }
            count = db.cchdo.find(ft).count()
            station = station + 1
            if count != 0:
                overlap = overlap + 1
                result.append(url)

100%|██████████| 115/115 [00:03<00:00, 36.27it/s]


In [8]:
station

33603

In [9]:
overlap

535

In [10]:
result.sort()

In [11]:
result

['/home/cz2397/data/cchdo-ctd/104_s04_nc_ctd/dis93_12355_00001_ctd.nc',
 '/home/cz2397/data/cchdo-ctd/108_pr14_g_nc_ctd/pr14_g_00004_00001_ctd.nc',
 '/home/cz2397/data/cchdo-ctd/108_pr14_g_nc_ctd/pr14_g_00016_00001_ctd.nc',
 '/home/cz2397/data/cchdo-ctd/111_ar11_d_nc_ctd/ar11_d_00020_00001_ctd.nc',
 '/home/cz2397/data/cchdo-ctd/111_ar11_d_nc_ctd/ar11_d_00022_00001_ctd.nc',
 '/home/cz2397/data/cchdo-ctd/117_s04i_nc_ctd/s04i_00001_00001_ctd.nc',
 '/home/cz2397/data/cchdo-ctd/117_s04i_nc_ctd/s04i_00107_00001_ctd.nc',
 '/home/cz2397/data/cchdo-ctd/118_18HU20010530_nc_ctd/18HU20010530_00084_00001_ctd.nc',
 '/home/cz2397/data/cchdo-ctd/122_33KB178_1_nc_ctd/prs2_00002_00001_ctd.nc',
 '/home/cz2397/data/cchdo-ctd/122_33KB178_1_nc_ctd/prs2_00002_00002_ctd.nc',
 '/home/cz2397/data/cchdo-ctd/122_33KB178_1_nc_ctd/prs2_00002_00003_ctd.nc',
 '/home/cz2397/data/cchdo-ctd/122_33KB178_1_nc_ctd/prs2_00002_00004_ctd.nc',
 '/home/cz2397/data/cchdo-ctd/122_33KB178_1_nc_ctd/prs2_00002_00005_ctd.nc',
 '/home